In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
from datetime import datetime

import dataProcess

In [2]:
# should use a better way to convert nans to numbers
def nan_to_zero(data):
  for i in range(data.shape[0]):
    for j in range(data.shape[1]):
      for k in range(data.shape[2]):
        if data[i, j, k].isnan:
          data[i, j, k] = 0

In [3]:
# data preprocessing

start = datetime(2024, 1, 3).date()
end = datetime(2024, 4, 11).date()
hourly_path ='weatherstats_toronto_hourly.csv'
daily_path ='weatherstats_toronto_daily.csv'

# data
data_tensor_preprocess = dataProcess.dataToTensorHourly(hourly_path, separateByDay=False, missingThreshold=0.1, columnToDelete=['wind_dir', 'unixtime'], start=start, end=end)
print(data_tensor_preprocess[0].shape)
# for some reason going back one day does not work, but this does
# need to go back one day so you're predicting the temperature for the next day
data = data_tensor_preprocess[0].reshape(-1, 24, 13)[:-1]
print(data.shape)
nan_to_zero(data)

# targets
targets = dataProcess.dailyTargets(daily_path, start=datetime(2024, 1, 4).date())
print(targets.shape)

torch.Size([2376, 13])
torch.Size([98, 24, 13])
torch.Size([98])


In [4]:
class WeatherPredictorTransformer(nn.Module):
  def __init__(self, input_size, d_model, nhead, num_encoder_layers, output_size):
    super(WeatherPredictorTransformer, self).__init__()
    self.input_size = input_size
    self.d_model = d_model
    self.embedding = nn.Linear(input_size, d_model)
    self.positional_encoding = self.create_sinusoidal_positional_encoding(24, d_model)
    self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
    self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers)
    self.output_linear = nn.Linear(d_model, output_size)

  def forward(self, x):
    # x: [batch_size, 24, input_size]
    x = self.embedding(x) # x: [batch_size, 24, d_model]
    x += self.positional_encoding[:]
    x = x.permute(1, 0, 2)  # x: [24, batch_size, d_model]
    encoder_output = self.transformer_encoder(x)
    prediction = self.output_linear(encoder_output[-1])
    return prediction

  def create_sinusoidal_positional_encoding(self, length, d_model):
    PE = torch.zeros(length, d_model)
    position = torch.arange(0, length, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
    PE[:, 0::2] = torch.sin(position * div_term)
    PE[:, 1::2] = torch.cos(position * div_term)
    return PE.unsqueeze(0)

In [5]:
def process_data_for_transformer(train_fraction, valid_fraction, data, targets, batch_size):

  train_dataset = TensorDataset(data, targets)
  total_size = len(data)
  train_split_point = int(total_size * train_fraction)
  valid_split_point = train_split_point + int(total_size * valid_fraction)

  train_dataset = TensorDataset(data[:train_split_point], targets[:train_split_point])
  val_dataset = TensorDataset(data[train_split_point:valid_split_point],
                              targets[train_split_point:valid_split_point])
  test_dataset = TensorDataset(data[valid_split_point:], targets[valid_split_point:])

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = process_data_for_transformer(0.8, 0.1, data, targets, 1)

In [9]:
input_size=13
d_model=512
nhead=8
num_encoder_layers=6
output_size=1

model = WeatherPredictorTransformer(input_size=input_size, d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers, output_size=output_size)

In [10]:
def train_model(model, num_epochs, optimizer, criterion, train_loader, val_loader, device):
  model = model.to(device)

  for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for inputs, targets in train_loader:
      inputs, targets = inputs.to(device).float(), targets.to(device).float()

      assert not torch.isnan(inputs).any(), "NaN found in inputs"
      assert not torch.isnan(targets).any(), "NaN found in targets"

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()
      total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}')

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
      for inputs, targets in val_loader:
        inputs, targets = inputs.to(device).float(), targets.to(device).float()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_model(model, 10, optimizer, criterion, train_loader, val_loader, device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Training Loss: 29.1682
Epoch [1/10], Validation Loss: 87.1941
Epoch [2/10], Training Loss: 25.1657
Epoch [2/10], Validation Loss: 90.9223
Epoch [3/10], Training Loss: 24.7273
Epoch [3/10], Validation Loss: 94.9328
Epoch [4/10], Training Loss: 24.3803
Epoch [4/10], Validation Loss: 98.2168
Epoch [5/10], Training Loss: 24.0223
Epoch [5/10], Validation Loss: 100.1720
Epoch [6/10], Training Loss: 23.9162
Epoch [6/10], Validation Loss: 101.7585
Epoch [7/10], Training Loss: 23.6827
Epoch [7/10], Validation Loss: 103.4656
Epoch [8/10], Training Loss: 23.6566
Epoch [8/10], Validation Loss: 105.1011
Epoch [9/10], Training Loss: 23.4752
Epoch [9/10], Validation Loss: 106.6330
Epoch [10/10], Training Loss: 23.2037
Epoch [10/10], Validation Loss: 107.2847
